<a href="https://colab.research.google.com/github/blaze515/ecomalyzer/blob/master/ecomalyzer2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GroupBy Ecommerce Analysis
Jake Dunn - Feb 22, 2020

## Load and Preprocess Data

In [0]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.stats import chi2, chi2_contingency, chisquare
import scipy.sparse as sp

In [2]:
data = pd.read_csv('/content/data-1.csv')
data.head()

,customer_integrated_id,refinement,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate
0,cjsyuvlck00013ra31lxcso69,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",3,0,0,0,0.0,0.0
1,cjwti47ac0001458du5ow5ikh,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",3,0,0,0,0.0,0.0
2,013146471,"{""name"":""visualVariant.nonvisualVariant.gbi_Me...",5,0,0,0,0.0,0.0
3,014182827,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",2,0,0,0,0.0,0.0
4,023424641,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",2,0,0,0,0.0,0.0


In [3]:
print(data.shape)

(168362, 8)


In [4]:
refinement_df = json_normalize(data['refinement'].apply(json.loads))
refinement_df.head()

,name,displayName,value,high,low
0,gbi_categories.1,Category,Vitamin C,NaN,NaN
1,visualVariant.nonvisualVariant.BRAND,Brands,Jason,NaN,NaN
2,visualVariant.nonvisualVariant.gbi_Merged_concern,Concern,Eye/Macular Health,NaN,NaN
3,visualVariant.nonvisualVariant.BRAND,Brands,Swanson Health Products,NaN,NaN
4,gbi_categories.1,Category,CoQ10,NaN,NaN


In [5]:
df = pd.concat([data, refinement_df], axis=1)
df.head(5)

,customer_integrated_id,refinement,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,name,displayName,value,high,low
0,cjsyuvlck00013ra31lxcso69,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",3,0,0,0,0.0,0.0,gbi_categories.1,Category,Vitamin C,NaN,NaN
1,cjwti47ac0001458du5ow5ikh,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",3,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Jason,NaN,NaN
2,013146471,"{""name"":""visualVariant.nonvisualVariant.gbi_Me...",5,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.gbi_Merged_concern,Concern,Eye/Macular Health,NaN,NaN
3,014182827,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",2,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Swanson Health Products,NaN,NaN
4,023424641,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",2,0,0,0,0.0,0.0,gbi_categories.1,Category,CoQ10,NaN,NaN


In [6]:
x_tab = pd.crosstab([df['displayName'], df['name']], columns='count')
x_tab

col_0                                                                                                  count
displayName                                        name                                                     
Brands                                             visualVariant.nonvisualVariant.BRAND                24149
Category                                           gbi_categories.1                                    80599
                                                   gbi_categories.2                                    40468
                                                   gbi_categories.3                                      831
Concern                                            visualVariant.nonvisualVariant.gbi_Merged_concern    4737
Flavor                                             visualVariant.nonvisualVariant.gbi_Merged_flavor      169
Form                                               visualVariant.nonvisualVariant.FORM                   616
                                                   visualVariant.nonvisualVariant.gbi_Merged_form       2603
More Ways to Browse                                visualVariant.nonvisualVariant.MOREWAYSTOBROWSE      5587
Pill Size                                          visualVariant.nonvisualVariant.PILLSZ                  34
Potency                                            visualVariant.nonvisualVariant.POTENT                2811
Specialty                                          visualVariant.nonvisualVariant.SDP                    564
                                                   visualVariant.nonvisualVariant.gbi_Merged_speci...   3666
Swanson Brands                                     visualVariant.nonvisualVariant.BRAND2                1474
visualVariant.nonvisualVariant.BAZARRATE           visualVariant.nonvisualVariant.BAZARRATE               25
visualVariant.nonvisualVariant.BAZARREVIEW         visualVariant.nonvisualVariant.BAZARREVIEW             24
visualVariant.nonvisualVariant.MOREWAYSTOBROWSE    visualVariant.nonvisualVariant.MOREWAYSTOBROWSE         4
visualVariant.nonvisualVariant.gbi_Merged_speci... visualVariant.nonvisualVariant.gbi_Merged_speci...      1

In [0]:
stat, p, dof, expected = chi2_contingency(x_tab)

In [8]:
print('dof=%d' % dof)
print(expected)

dof=0
[[2.4149e+04]
 [8.0599e+04]
 [4.0468e+04]
 [8.3100e+02]
 [4.7370e+03]
 [1.6900e+02]
 [6.1600e+02]
 [2.6030e+03]
 [5.5870e+03]
 [3.4000e+01]
 [2.8110e+03]
 [5.6400e+02]
 [3.6660e+03]
 [1.4740e+03]
 [2.5000e+01]
 [2.4000e+01]
 [4.0000e+00]
 [1.0000e+00]]


In [9]:
prob = .95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))

probability=0.950, critical=nan, stat=0.000


In [10]:
chisquare(x_tab)

Power_divergenceResult(statistic=array([772723.86695335]), pvalue=array([0.]))

In [11]:
df.head()

,customer_integrated_id,refinement,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,name,displayName,value,high,low
0,cjsyuvlck00013ra31lxcso69,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",3,0,0,0,0.0,0.0,gbi_categories.1,Category,Vitamin C,NaN,NaN
1,cjwti47ac0001458du5ow5ikh,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",3,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Jason,NaN,NaN
2,013146471,"{""name"":""visualVariant.nonvisualVariant.gbi_Me...",5,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.gbi_Merged_concern,Concern,Eye/Macular Health,NaN,NaN
3,014182827,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",2,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Swanson Health Products,NaN,NaN
4,023424641,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",2,0,0,0,0.0,0.0,gbi_categories.1,Category,CoQ10,NaN,NaN


In [12]:
df.describe()

,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,high,low
count,168362.000000,168362.000000,168362.000000,168362.000000,168362.000000,168362.000000,49.000000,49.000000
mean,4.083730,1.692448,0.397091,0.149214,0.145661,0.395862,48982.142857,12.142857
std,5.276081,3.614590,2.150449,0.582416,0.427392,0.507926,50505.101846,7.778175
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,4.600000
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,4.600000
50%,3.000000,1.000000,0.000000,0.000000,0.000000,0.333333,5.000000,4.600000
75%,4.000000,2.000000,0.000000,0.000000,0.000000,0.600000,100000.000000,20.000000
max,347.000000,467.000000,512.000000,64.000000,33.166667,18.000000,100000.000000,20.000000


In [13]:
df.isnull().sum()

customer_integrated_id         0
refinement                     0
searches                       0
viewProducts                   0
addToCarts                     0
orders                         0
conversion_rate                0
clickthrough_rate              0
name                           0
displayName                    0
value                         49
high                      168313
low                       168313
dtype: int64

In [14]:
df['value'].fillna('', inplace=True)
df['high'].fillna(df['high'].mean(), inplace=True)
df['low'].fillna(df['low'].mean(), inplace=True)
df.isnull().sum()

customer_integrated_id    0
refinement                0
searches                  0
viewProducts              0
addToCarts                0
orders                    0
conversion_rate           0
clickthrough_rate         0
name                      0
displayName               0
value                     0
high                      0
low                       0
dtype: int64

In [15]:
def join_feature(row):
  return ("%s_%s" % (row['displayName'], row['value']))

df['combined'] = df.apply(join_feature, axis=1)
df.head(5)

,customer_integrated_id,refinement,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,name,displayName,value,high,low,combined
0,cjsyuvlck00013ra31lxcso69,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",3,0,0,0,0.0,0.0,gbi_categories.1,Category,Vitamin C,48982.142857,12.142857,Category_Vitamin C
1,cjwti47ac0001458du5ow5ikh,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",3,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Jason,48982.142857,12.142857,Brands_Jason
2,013146471,"{""name"":""visualVariant.nonvisualVariant.gbi_Me...",5,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.gbi_Merged_concern,Concern,Eye/Macular Health,48982.142857,12.142857,Concern_Eye/Macular Health
3,014182827,"{""name"":""visualVariant.nonvisualVariant.BRAND""...",2,0,0,0,0.0,0.0,visualVariant.nonvisualVariant.BRAND,Brands,Swanson Health Products,48982.142857,12.142857,Brands_Swanson Health Products
4,023424641,"{""name"":""gbi_categories.1"",""displayName"":""Cate...",2,0,0,0,0.0,0.0,gbi_categories.1,Category,CoQ10,48982.142857,12.142857,Category_CoQ10


In [0]:
combined = df['combined'].to_dict()

In [17]:
trimmed_df = df.drop(columns=['refinement', 'name', 'displayName', 'combined','value'])

trimmed_df

,customer_integrated_id,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,high,low
0,cjsyuvlck00013ra31lxcso69,3,0,0,0,0.000000,0.000000,48982.142857,12.142857
1,cjwti47ac0001458du5ow5ikh,3,0,0,0,0.000000,0.000000,48982.142857,12.142857
2,013146471,5,0,0,0,0.000000,0.000000,48982.142857,12.142857
3,014182827,2,0,0,0,0.000000,0.000000,48982.142857,12.142857
4,023424641,2,0,0,0,0.000000,0.000000,48982.142857,12.142857
...,...,...,...,...,...,...,...,...,...
168357,012228043,41,24,2,0,0.048780,0.585366,48982.142857,12.142857
168358,021537894,41,1,2,0,0.048780,0.024390,48982.142857,12.142857
168359,023694769,138,134,274,0,1.985507,0.971014,48982.142857,12.142857
168360,cjtdjeyg200013aazf2cobwf1,73,3,5,4,0.123288,0.041096,48982.142857,12.142857


In [18]:
count_vect = CountVectorizer(stop_words='english')
# id_counts = count_vect.fit_transform(trimmed_df['customer_integrated_id'].to_numpy())
# trimmed_df2 = trimmed_df.drop(columns=['customer_integrated_id'])
# x = trimmed_df2.to_numpy()
# x.shape
# train = sp.hstack(trimmed_df.apply(lambda col: count_vect.fit_transform(col)))
# x = sp.hstack([id_counts, x])
# x.shape
# x_train
# id_counts.shape
# count_vect.get_feature_names()
# combined.data
# y = count_vect.transform(combined)
y = count_vect.fit_transform(df['combined'].to_numpy())
# count_vect.get_feature_names()
y.shape
# y.shape

(168362, 2470)

In [0]:
# dict_vect = DictVectorizer()
# records = trimmed_df.drop(columns=['customer_integrated_id']).to_dict('records')
# records
# x = dict_vect.fit_transform(records)
# x.shape
# # dict_vect.get_feature_names()

# y = dict_vect.transform(combined)
# y

In [20]:
print(len(df))
df['displayName'].unique()

smaller = df[df['displayName'].isin(['Category', 'Brands', 'Concern', 'Specialty'])]
print(len(smaller))
smaller['displayName'].unique()
smaller.head()
# df.describe()
# y = df['combined']
test = smaller.drop(columns=['value', 'name', 'displayName', 'refinement', 'customer_integrated_id'])
y = smaller['customer_integrated_id']
test.head()

168362
155014


,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,high,low,combined
0,3,0,0,0,0.0,0.0,48982.142857,12.142857,Category_Vitamin C
1,3,0,0,0,0.0,0.0,48982.142857,12.142857,Brands_Jason
2,5,0,0,0,0.0,0.0,48982.142857,12.142857,Concern_Eye/Macular Health
3,2,0,0,0,0.0,0.0,48982.142857,12.142857,Brands_Swanson Health Products
4,2,0,0,0,0.0,0.0,48982.142857,12.142857,Category_CoQ10


In [21]:
x = pd.get_dummies(test)
x

,searches,viewProducts,addToCarts,orders,conversion_rate,clickthrough_rate,high,low,combined_Brands_123 Gluten Free,combined_Brands_21st Century,combined_Brands_A.C. Grace Company,combined_Brands_Abra Therapeutics,combined_Brands_Absolute Nutrition,combined_Brands_ActiPet,combined_Brands_Acure,combined_Brands_Advanced Nutrition By Zahler,combined_Brands_Advanced Research/Nutrient Carriers,combined_Brands_Aerobic Life,combined_Brands_Ageless Foundation,combined_Brands_Airborne,combined_Brands_Ajipure,combined_Brands_AkPharma Inc.,combined_Brands_Alacer Emergen-C,combined_Brands_Alaffia,combined_Brands_Alba Botanica,combined_Brands_Align,combined_Brands_Alka-Seltzer,combined_Brands_Alkalife,combined_Brands_All One,combined_Brands_Allergy Research Group,combined_Brands_Allimax,combined_Brands_Allmax Nutrition,combined_Brands_Almased,combined_Brands_Aloe Cadabra,combined_Brands_Aloe Life,combined_Brands_Aloha Bay,combined_Brands_Alta Health Products,combined_Brands_Alvera,combined_Brands_Alvita Tea,combined_Brands_Amazing Grass,...,combined_Concern_Urinary Tract Infection,combined_Concern_Vegan,combined_Concern_Vegetarian,combined_Concern_Weight Gain,combined_Concern_Weight Loss,combined_Concern_Womens Health,combined_Concern_Wrinkles,combined_Concern_Yeast Infection,combined_Concern_pH Balanced,combined_Specialty_Alcohol Free,combined_Specialty_BPA Free,combined_Specialty_Biodegradable,combined_Specialty_Caffeine Free,combined_Specialty_Certified Organic,combined_Specialty_Cruelty Free,combined_Specialty_Dairy Free,combined_Specialty_Fat Free,combined_Specialty_Fragrance Free,combined_Specialty_Gluten Free,combined_Specialty_Grain Free,combined_Specialty_Hypoallergenic,combined_Specialty_Kosher,combined_Specialty_Lactose Free,combined_Specialty_Non GMO,combined_Specialty_Non GMO Project Verified,combined_Specialty_Non Toxic,combined_Specialty_Non-GMO,combined_Specialty_Non-GMO Project Verified,combined_Specialty_Organic,combined_Specialty_Paraben Free,combined_Specialty_Peanut Free,combined_Specialty_Shellfish Free,combined_Specialty_Soy Free,combined_Specialty_Standardized,combined_Specialty_Sugar Free,combined_Specialty_Tree Nut Free,combined_Specialty_Unsweetened,combined_Specialty_Vegan,combined_Specialty_Vegetarian,combined_Specialty_pH Balanced
0,3,0,0,0,0.000000,0.000000,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,0,0,0,0.000000,0.000000,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0.000000,0.000000,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0.000000,0.000000,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0.000000,0.000000,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168357,41,24,2,0,0.048780,0.585366,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
168358,41,1,2,0,0.048780,0.024390,48982.142857,12.142857,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [22]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.33, random_state=42)
y_train

31577                     013429423
12412     cjvwnji4p00013h9ldnz5o4pv
66        cjwu7s1o000013v9k6l39ldif
6267      cjw2xxwo300013oagxm05yefs
92475                     021269515
                    ...            
130730    cjsw9kiz900013aaxcofidk4i
112995                    023751711
143666    cjs3lrpyk00013mayedqcs3qp
159613    cjolfuqxz00013mb7u9wtx086
132916    cjtay2yb800013bb7pmh99gms
Name: customer_integrated_id, Length: 103859, dtype: object

In [0]:
# from sklearn.feature_extraction.text import TfidfTransformer 
# tfidf_transformer = TfidfTransformer()
# y_train_tfidf = tfidf_transformer.transform(y_train)
# y_train_tfidf.shape

# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# X = vectorizer.transform(y_train)
# print(vectorizer.get_feature_names())
# X.shape

## Train Models



In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train, y_train)
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_matrix = cosine_similarity(y)

In [0]:
predicted = clf.predict(x_test)
np.mean(predicted == y_test)

In [0]:
# from sklearn import svm
# svm_clf = svm.SVC()
# svm_clf.fit(x_train, y_train)

In [0]:
# predicted2 = svm_clf.predict(x_test)
# np.mean(predicted2 == y_test)